# Use case 1: 

# Breast cancer dataset

# Setup

- Input: Application with the python files, data set and requirements.
- Output: Platform.

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:

from autodeploy.setup import setup


app_dir = '/home/guess/Desktop/autodeploy/examples/leaf'
workflow = {'gathering': 'gathering.py',
            'preprocessing': 'preprocessing.py',
            'modeling': 'modeling.py',
            'main': 'main.py'}

platform = setup.Setup(app_dir, workflow)
platform

In [2]:
platform.pipeline()
platform

23-Oct-19 21:52:41 - Pipeline: build() and run() are running.
23-Oct-19 21:52:41 - Dockerfile was found.
23-Oct-19 21:52:41 - 404 Client Error: Not Found ("no such image: app_single: No such image: app_single:latest")
23-Oct-19 21:52:41 - Container creation failed.
23-Oct-19 21:53:18 - Image app_single was built successfully.
23-Oct-19 21:53:19 - MLproject was found.
23-Oct-19 21:53:19 - Image app_single is running as app_single container.
23-Oct-19 21:53:20 - MLflow server is running at 0.0.0.0:8001


In [9]:
# platform.stop()

23-Oct-19 21:50:46 - Container app_single was stopped.
23-Oct-19 21:50:46 - Stopped containers were deleted.


# Deploy

- Input: Platform object.
- Output: Workflow shown at Mlflow webserver http://localhost:8001/#/. API model at http://localhost:5001/#/

In [5]:
from autodeploy.deploy import deploy

deployer = deploy.Deploy(platform)

deployer.pipeline()

23-Oct-19 21:58:37 - Pipeline: run_workflow() and deploy() are running.
23-Oct-19 21:58:37 - Running workflow: type=single .
23-Oct-19 21:58:42 -  Main file (main.py) output:  Run matched, but is not FINISHED, so skipping (run_id=e48d762176914bfa807563d6dba8bc5c, status=FINISHED)
No matching run has been found.
2019/10/23 19:58:38 INFO mlflow.projects: === Created directory /tmp/tmp5koyfqp6 for downloading remote URIs passed to arguments of type 'path' ===
2019/10/23 19:58:38 INFO mlflow.projects: === Running command 'python gathering.py' in run with ID '8fd33bc89be1452e8ea568fbdf4f9fc7' === 
   species  specimen_number  eccentricity  ...  third_moment  uniformity  entropy
0        1                1       0.72694  ...      0.005232    0.000275  1.17560
1        1                2       0.74173  ...      0.002708    0.000075  0.69659
2        1                3       0.76722  ...      0.000921    0.000038  0.44348
3        1                4       0.73797  ...      0.001154    0.000066

## Track


- Input: Values to predict.
- Output: Predictions and analysis.

In [6]:
from autodeploy.deploy import deploy

deployer = deploy.Deploy(api_image_name='app_single_api')
deployer.run_api()


23-Oct-19 22:02:53 - Creating container: app_single_api. 
23-Oct-19 22:02:53 - 409 Client Error: Conflict ("Conflict. The container name "/app_single_api" is already in use by container "945e5118a50a4200c0666d425983b3a64474b132f41ba75791cc7025b050da74". You have to remove (or rename) that container to be able to reuse that name.")
23-Oct-19 22:02:53 - Container creation failed.


In [3]:
from sklearn.datasets import make_classification
from autodeploy.track import track

tracker = track.Track(port=5001)

X2, y2 = make_classification(n_samples=100,n_features=14,
                            class_sep=1.1, n_informative=2,
                            n_redundant=1, random_state=42)
sample_input = { 
        "columns": [ f"x_{i}" for i in range(14)], 
        "data": X2.tolist()
        # "data": [ 
        #     [-1.8873649,-1.1455,0.83967,-2.008855],
        #     [-0.989919,  0.987401, 0.266892,  0.485329]  
        # ] 
}   

# print(sample_input)
path = '/home/guess/Desktop/autodeploy/examples/leaf'
predictions = tracker.predict(sample_input, to_save=True, path_pred=path)
# tracker.save_prediction(path)

predictions.head()
# print(f'Predictions: \n {predictions}')

23-Oct-19 22:11:53 -  INFO - Predicting from port: 5001
23-Oct-19 22:11:53 -  INFO - Time elapsed: 0.012650012969970703
23-Oct-19 22:11:53 -  INFO - Input and predictions were saved at: /home/guess/Desktop/autodeploy/examples/leaf/predictions.csv


,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11,x_12,x_13,pred
0,0.334457,-0.798297,0.658544,1.956055,-0.176947,2.010205,-1.379319,1.575381,-0.387702,0.285865,-0.730930,-0.153198,1.794558,-0.033127,6
1,0.018418,0.829406,1.676437,-0.489198,-0.219101,0.326927,-2.211135,-0.901859,-1.870792,-0.351513,0.235615,1.605752,-1.478586,0.770865,24
2,-0.704344,-1.280429,-1.408461,-0.633558,0.606010,-1.556629,1.754794,-1.024597,-0.522723,1.049009,-2.081929,1.637036,0.211017,1.696456,10
3,-0.557492,-0.803675,-1.369803,1.255093,2.579709,-0.088282,1.639117,0.507432,0.963879,2.210523,1.677701,1.455370,0.568983,-0.553588,10
4,-0.203580,-1.223940,-0.581681,1.352911,-0.649278,-1.014757,0.034083,0.666845,1.628397,-0.379128,-0.769973,1.198853,-1.555896,0.233786,23


In [17]:
# %store predictions

### Gather a sample from the training data and new distribution


It aims at checking the sanity of new data; basically, test integrity (same numbers of features, class labels, types, etc.) and draft distributions (tests, e.g. Kolmogorov.).

In [6]:
# SAVE TRAINING X, TRAINING y
import pandas as pd
from autodeploy.track import tools

old_data_path = '/home/guess/Desktop/autodeploy/examples/leaf/workflow/gathered.csv'
X_old = pd.read_csv(old_data_path)
X_old = X_old.loc[:, X_old.columns != 'species']
X_old = X_old.loc[:, X_old.columns != 'species']
X_new = predictions.loc[:, predictions.columns != 'pred']

tools.overall_test(X_old.sample(len(X_new)), X_new, verbose=True)


# X_old.head()
# assert integrity_checker(X_old) == integrity_checker(X_new)
# tracker.overall_test(X_old, X_new)

########## Comparing two numerical dataframes ##########

######## Kolmogorov-Smirnov test ########


KeyError: 'specimen_number'

# Comparison

# Mlflow vs autodeploy
| Features             | MLflow                                                 | autodeploy                                                               |
|----------------------|--------------------------------------------------------|--------------------------------------------------------------------------|
| Track of experiments | Any stage in the ML workflow with logs and artifacts. | Based on MLflow but with some built-in  functions to standard workflows (single, cluster). |
| Reproduce code       | Conda environment and Dockerfile  manually filled.     | User provides basic config (requirements) and the environment is created automatically |
| Deploy models        | Built-in functions, no tracking.                                | Based on MLflow with add-ons tools  to track the deployed models.        |
| Replication          | It is handled by hand.                       | Specify the number of replicas in the  deployment stage.                 |
| Learning curve       | Medium, Python and  command line scripts.              | Easy, everything is written in Python.                                   |

Similar to the relationship between Tensorflow and Keras

## Why is it useful?

- One of the hardest part of ML productionalizing lays on making the model easy to track, easy to deploy and easy to maintain. That's why having tools that lead to rapid prototyping and easy interpretation is crucial.

- Due to all the output work is based on Docker, it will be easily deploy in any environment, whether on-premise or cloud.

- **The main goal is to automate the model's life after modeling. It means, automate the post-modeling stage.**


## Unique features for docker container that contains ML models.

# Plugins

## Test data augmentation: generate new feedbacks

In [ ]:

new_feedbacks = tracker.generate_feedbacks(feedbacks)

augmented_data = X_old+new_feedbacks
model.train(augmented_data)

new_predictions = model.predict(X_hidden)
print(model.evaluate(new_predictions , y_hidden))


### If the new model is better then the data augmentation has worked succesfully.

Old score = 0.81

New score = 0.83

### TODO


- Show predictions in a better way.
- Comparison with mlflow.
- helping in a data analysis
- Replication when using several containers
- Change position of mlruns (artifacts)
- Change loginfo deploy (5001)